In [1]:
import pandas as pd
import numpy as np
import json 
import copy
import csv
import os
import re
import nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
from nltk.corpus import stopwords
import sklearn
from ngram import *
from nltk.stem import PorterStemmer

In [2]:
data = [] 
cf = open('bars_rev_tr_df.csv','r')

file = csv.DictReader(cf)
#print(file.fieldnames)

for x in file:
    line = {'business_id':x['business_id'],'date':x['date'],'stars':x['stars'],'text':x['text']}
    data.append(line)

cf.close()

In [3]:
id = open('uniq_id_lvbars.txt','r')
id_str=id.read()
id.close()

In [4]:
id_list=id_str.split(',')

In [5]:
dat_night=[]
for i in range(len(data)):
    if data[i]['business_id'] in id_list:
        dat_night.append(data[i])

In [6]:
dat_night_5star=[]
dat_night_4star=[]
dat_night_3star=[]
dat_night_2star=[]
dat_night_1star=[]
for i in range(len(dat_night)):
    if dat_night[i]['stars']=='5':
        dat_night_5star.append(dat_night[i])
    elif dat_night[i]['stars']=='4':
        dat_night_4star.append(dat_night[i])
    elif dat_night[i]['stars']=='3':
        dat_night_3star.append(dat_night[i])
    elif dat_night[i]['stars']=='2':
        dat_night_2star.append(dat_night[i])
    else:
        dat_night_1star.append(dat_night[i])



In [12]:
reviewslist = copy.deepcopy(dat_night_1star)
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\$)|(\*)|(\%)|(\_)|(\=)|(\#)|(\&)|(\~)|(\@)")#[^\P{P}-]+
REPLACE_WITH_SPACE = re.compile("(\n)|(\-)|(\/)|(\d)")

def preprocess_reviews(reviews):
    reviews = REPLACE_NO_SPACE.sub("", reviews)
    reviews = REPLACE_WITH_SPACE.sub(" ", reviews)
    return reviews

reviewsTEXT_clean = copy.deepcopy(reviewslist)

text_cloud1=''

for ind in range(len(reviewslist)):
    texts = ''
    texts = reviewslist[ind]['text']
    texts = texts.lower()
    
    texts = re.sub('n\'t',' not', texts)
    
    texts = re.sub('isnt','isn\'t', texts)
    texts = re.sub('wasnt','wasn\'t', texts)
    texts = re.sub('werent','weren\'t', texts)
    texts = re.sub('dont','don\'t', texts)
    texts = re.sub('doesnt','doesn\'t', texts)
    texts = re.sub('didnt','didn\'t', texts)
    texts = re.sub('hasnt','hasn\'t', texts)
    texts = re.sub('havent','haven\'t', texts)
    texts = re.sub('hadnt','hadn\'t', texts)
    texts = re.sub('mightnt','mightn\'t', texts)
    texts = re.sub('shouldnt','shouldn\'t', texts)
    texts = re.sub('isn','isn\'t', texts)
    texts = re.sub('wasn','wasn\'t', texts)
    texts = re.sub('weren','weren\'t', texts)
    texts = re.sub('don','don\'t', texts)
    texts = re.sub('doesn','doesn\'t', texts)
    texts = re.sub('didn','didn\'t', texts)
    texts = re.sub('hasn','hasn\'t', texts)
    texts = re.sub('haven','haven\'t', texts)
    texts = re.sub('hadn','hadn\'t', texts)
    texts = re.sub('mightn','mightn\'t', texts)
    texts = re.sub('shouldn','shouldn\'t', texts)
    texts = re.sub('won\'t','will not', texts)
    
    texts = re.sub('n\'t',' not', texts)
    
    #add NOT_
    pattern = r'\.|\;|\!|\?|\,|\)|\(|\:|\'|\"|\%'
    list_text=re.split(pattern,texts)
    
    sent=''
    for i in range(len(list_text)):
        list_text[i] = re.sub('\+','', list_text[i])
        list_text[i] = re.sub('\*','', list_text[i])
        list_text[i] = re.sub('\$','', list_text[i])
        list_text[i] = re.sub('\[','', list_text[i])
        list_text[i] = re.sub('\]','', list_text[i])
        list_text[i] = re.sub('\%','', list_text[i])
        list_text[i] = re.sub('\\\\',' ', list_text[i])
        matchObj1 = re.search(r'(.*)not (.*)',list_text[i])
        matchObj2 = re.search(r'(.*) never (.*)',list_text[i])
        
        if matchObj1 != None :
            sub=re.sub(r' ', " notadd", ' '+matchObj1.group(2))
            list_text[i] = re.sub(matchObj1.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'not ','', list_text[i],1)
            #list_text[i] = re.sub(',',' ', matchObj1.group(3))
            sent = sent + list_text[i] + ' '
        
        elif matchObj2 != None :
            sub=re.sub(r' ', " notadd", ' '+matchObj2.group(2))
            list_text[i] = re.sub(matchObj2.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'never ','', list_text[i],1)
            #list_text[i] = re.sub(',',' ', matchObj1.group(3))
            sent = sent + list_text[i] + ' '
        
        else:
            sent = sent + list_text[i] + ' '
   
    #print(sent)
    #print(texts)
    #print(texts)
    text_clean=preprocess_reviews(sent)
    #print(text_clean)
    
    text_cloud1=text_cloud1+text_clean
    reviewsTEXT_clean[ind]['text']=text_clean
    reviewsTEXT_clean[ind]['text_split']=text_clean.split()
    reviewsTEXT_clean[ind]['bigrams']=generate_ngrams(text_clean, 5)
    reviewsTEXT_clean[ind]['freq']=nltk.FreqDist(reviewsTEXT_clean[ind]['text_split'])
    reviewsTEXT_clean[ind]['bigrams_freq']=nltk.FreqDist(reviewsTEXT_clean[ind]['bigrams'])
    
#reviewsTEXT_clean[0]

In [13]:
#select words due to frequency order
AllFreq = reviewsTEXT_clean[0]['freq']
for ind in range(len(reviewsTEXT_clean)-1):
    freqs = reviewsTEXT_clean[ind+1]['freq']
    AllFreq.update(freqs)
    
wordList = sorted(AllFreq.items(),key=lambda item:item[1],reverse=True)

#choose the important ones--No.1-100
WORDS=[x[0] for x in wordList]
WORDS=WORDS[0:1000]

In [14]:
stop_words = set(stopwords.words('english')) 
negWords=set(["wouldn't",'isn','wasn',"weren't", "haven't", "hasn't", "couldn't", "isn't", 'doesn','hasn',"mustn't", 'mightn', 'shan', 'no', "wasn't",'aren', "didn't", "hadn't","don't",'nor',"won't",'weren',"doesn't","needn't", 'shouldn',"mightn't","shan't", 'wouldn',"shouldn't",'hadn'])
d=set(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','à','{sigh}'])
stopWords=stop_words-negWords
stopWords.update(d)

WORDS=[x for x in WORDS if x not in stopWords]

In [104]:
#select bigrams due to frequency order
AllBiFreq = reviewsTEXT_clean[0]['bigrams_freq']
for ind in range(len(reviewsTEXT_clean)-1):
    Bifreqs = reviewsTEXT_clean[ind+1]['bigrams_freq']
    AllBiFreq.update(Bifreqs)
    
BiwordList = sorted(AllBiFreq.items(),key=lambda item:item[1],reverse=True)

#choose the important ones--No.1-15000
BiWORDS=[x[0] for x in BiwordList]

In [119]:
BiwordList=BiwordList[0:3000]

In [120]:
Biwordfreq = {}
for i in range(len(BiwordList)):
    if len(list(set(BiwordList[i][0].split()).difference(stopWords))) >= 2:
        Biwordfreq[BiwordList[i][0]]=BiwordList[i][1]

In [ ]:
'''
'beer selection': 72,
'draft beer': 64,
'beer pong': 52,'craft beer':
'''

In [1]:
a='Poor beer selection and horrible job filtering out the cigarette smoke. I couldn\'t breathe in there.'

In [2]:
a

"Poor beer selection and horrible job filtering out the cigarette smoke. I couldn't breathe in there."

In [7]:
pre_list=[]
if 'beer selection' in a:
    print(a)
    temp_list=a.split()
    for i in range(len(temp_list)):
        if ((temp_list[i]=='beer') and (temp_list[i+1]=='selection')):
            pre_list.append(temp_list[i-1])
        elif ((temp_list[i]=='beer') and (temp_list[i+1]=='selection') and (temp_list[i+2]=='is')):
            pre_list.append(temp_list[i+2])
    print('--------')
pre_list    


Poor beer selection and horrible job filtering out the cigarette smoke. I couldn't breathe in there.
--------


['Poor']

In [23]:
for i in range(len(dat_night_1star)):
    if 'beer selection' in dat_night_1star[i]['text']:
        temp=dat_night_1star[i]['text']
        
        print(dat_night_1star[i]['text'])
        print('--------')

I've been to PS 702 a number of times for the experience and beer selection. Understanding the service is usually terrible, but Kaila at your Las Vegas location took it a step further. Took roughly 15 min to get two drinks, then took another 25 min to get our meal(my salmon came out overdone and 15 min before my GFs poke came out). I am used to both entrees coming out at the same time and the waitress not giving attitude when asked what issue occurred. Mark, the alleged manager was very nice and took our drinks off of the bill, but Kaila was noticably rude throughout the experience. Kaila is the epitome of what not to be in regards to Las Vegas industry standards. I'll no longer frequent this location again. 

Lazy dogs and Wolfgang Puck's next door has never steered me wrong though.
--------
There's no zero star rating here.  I received a mailer from them which had a coupon for a beer plus an appetizer for $2.95.  The bartender on duty said I'd have to gamble $100 for the coupon to be

In [159]:
df_ngram=pd.DataFrame(columns=list(Biwordfreq.keys()))

In [145]:
'''
def word_count_in_str(string, keyword):
    return len(string.split(keyword))-1
for i in range(len(dat_night_1star)):
    print(i)
    list_temp=[]
    for j in range(len(list(Biwordfreq.keys()))):    
        list_temp.append(word_count_in_str(reviewsTEXT_clean[i]['text'], list(Biwordfreq.keys())[j]))
    df_ngram.loc[i]=list_temp
        #df_ngram.iloc[i,j] = word_count_in_str(reviewsTEXT_clean[i]['text'], list(Biwordfreq.keys())[j])
'''